In [1]:
from pathlib import Path
import os
import pandas as pd
import pickle

In [2]:
clean_data_folder = Path('../clean_data')
save_folder = Path('../data_for_model')
version_name = 'data_for_model_v1'

In [3]:
df = pd.read_parquet(clean_data_folder/'enriched_dtb_hk.parquet')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148406 entries, 0 to 148405
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   mssv     148406 non-null  category
 1   hoc_ky   148406 non-null  category
 2   nam_hoc  148406 non-null  category
 3   dtb_hk   148140 non-null  float32 
dtypes: category(3), float32(1)
memory usage: 1.8 MB


In [5]:
data_for_model = {
    'input': [],
    'target': [],
    'mo_ta': 
f'''
==================     VERSION {version_name}     ==================
* target là dtb_hk của 1 sinh viên trong 2 học kỳ liên tiếp
* input là tất cả dtb_hk của sv đó trong các học kỳ trước (target)
* diem được normalize từ [0-10] về [0-1]
LƯU Ý:
* input và target được sắp xếp theo thứ tự học kỳ tăng dần
* Các học kỳ có giá trị null đã được lại bỏ. 
  Xem như học kỳ trước và sau của học kỳ null đó là 2 học kỳ liên tiếp
* input và target chưa được shuffle
'''
}

In [6]:
df.isnull().any(axis=1).sum()

266

In [7]:
index = df[df.isnull().any(axis=1)].index
df.drop(index=index, inplace=True)

In [8]:
df.isnull().any(axis=1).sum()

0

In [9]:
df.dtb_hk = df.dtb_hk / 10

In [10]:
df.dtb_hk.describe()

count    148140.000000
mean          0.640012
std           0.209803
min           0.000000
25%           0.557692
50%           0.690909
75%           0.785714
max           1.000000
Name: dtb_hk, dtype: float64

In [11]:
group_df = df.sort_values(by=['nam_hoc', 'hoc_ky']).groupby('mssv')

In [12]:
group_df.get_group('0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6')

,mssv,hoc_ky,nam_hoc,dtb_hk
69754,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,1,2021,0.780769
69755,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,2,2021,0.827500
69756,0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6,1,2022,0.750833


In [13]:
sv_dtb_hk = group_df.apply(lambda x: tuple(x.dtb_hk))
sv_dtb_hk

mssv
0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6    (0.7807692289352417, 0.8274999856948853, 0.750...
0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G    (0.889090895652771, 0.8912500143051147, 0.9053...
00046394XPVAIBAEXE+FMXCQGVRIBECT4YMJHSFD    (0.5277777910232544, 0.7124999761581421, 0.790...
00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD    (0.5277777910232544, 0.7124999761581421, 0.790...
0006A0BBXPvAibaEXe/lMOwHQdw54DgUkWaqwb1u                                 (0.721818208694458,)
                                                                  ...                        
FFF3D630XPvAibaEXe8byJxgVsE7R2dP7ICEdFWK    (0.90666663646698, 0.7772727608680725, 0.80789...
FFF4CD57XPvAibaEXe/z8kRiyNmKNla425pm0Qmc        (0.7662500143051147, 0.764285683631897, 0.75)
FFFA4234XPVAIBAEXE+OHMJSJ4XEZDTGGCTDT9FT             (0.7493332624435425, 0.5199999809265137)
FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT    (0.7421427965164185, 0.6786667108535767, 0.751...
FFFCEDACXPvAibaEXe/r1+Wy29gBaT8F536ZLuBZ    (0.59250003

In [14]:
data_for_model['input'] = []
data_for_model['target'] = []
for dtb_hk in sv_dtb_hk:
    # print('haha', dtb_hk)
    for i in range(1, len(dtb_hk) +1 - 2):
        # print(dtb_hk[:i], dtb_hk[i:i+2])
        data_for_model['input'].append(dtb_hk[:i])
        data_for_model['target'].append(dtb_hk[i:i+2])

In [15]:
data_for_model.keys()

dict_keys(['input', 'target', 'mo_ta'])

In [16]:
print(data_for_model['mo_ta'])


==================     VERSION data_for_model_v1     ==================
* target là dtb_hk của 1 sinh viên trong 2 học kỳ liên tiếp
* input là tất cả dtb_hk của sv đó trong các học kỳ trước (target)
* diem được normalize từ [0-10] về [0-1]
LƯU Ý:
* input và target được sắp xếp theo thứ tự học kỳ tăng dần
* Các học kỳ có giá trị null đã được lại bỏ. 
  Xem như học kỳ trước và sau của học kỳ null đó là 2 học kỳ liên tiếp
* input và target chưa được shuffle



In [17]:
with open(save_folder/f'{version_name}.pkl', 'wb') as file:
    pickle.dump(data_for_model, file)